In [1]:
import pandas as pd
import numpy as np
import pyecharts 

In [2]:
data = pd.read_csv('patient_data.csv', engine='python') #옵션 써야 한글 읽힘

In [3]:
data = data.fillna("")

In [4]:
data.head()

,date,ID,degree,hospital,condition,sex,type,age,country,note
0,2020-01-19,1,1차,인천의료원,퇴원,여,중국인 관광객,30대,중국,
1,2020-01-22,2,1차,국립중앙의료원,퇴원,남,우한 교민,50대,한국,
2,2020-01-25,3,1차,일산명지병원,퇴원,남,우한 교민,50대,한국,
3,2020-01-26,4,1차,분당서울대병원,퇴원,남,우한 방문자,50대,한국,
4,2020-01-30,5,1차,서울의료원,입원,남,우한 방문자,30대,한국,


In [5]:
data2 = data.groupby(['date', 'degree']).size()
data2 = data2.reset_index()
data2.columns = ['date', 'degree', 'count']
data2.date = pd.to_datetime(data2['date'], format="%Y-%m-%d")
data2 = data2.set_index("date")
data2.head()

,degree,count
date,,
2020-01-19,1차,1
2020-01-22,1차,1
2020-01-25,1차,1
2020-01-26,1차,1
2020-01-30,1차,2


In [6]:
data3 = pd.DataFrame({'date' : pd.date_range("2020-01-19", "2020-02-17")})

In [7]:
data_mer = pd.merge(data3, data2, how='left', on='date')

In [8]:
data_mer.head()

,date,degree,count
0,2020-01-19,1차,1.0
1,2020-01-20,NaN,NaN
2,2020-01-21,NaN,NaN
3,2020-01-22,1차,1.0
4,2020-01-23,NaN,NaN


In [9]:
count1 = []
for x in range(data_mer.shape[0]):
    if data_mer.loc[x, 'degree'] == "1차":
        count1.append(data_mer.loc[x,'count'] * 1)
    else: count1.append(data_mer.loc[x,'count'] * 0)
data_mer['count_1'] = list(count1)

In [10]:
count2 = []
for x in range(data_mer.shape[0]):
    if data_mer.loc[x, 'degree'] == "2차":
        count2.append(data_mer.loc[x,'count'] * 1)
    else: count2.append(data_mer.loc[x,'count'] * 0)
data_mer['count_2'] = list(count2)

In [11]:
count3 = []
for x in range(data_mer.shape[0]):
    if data_mer.loc[x, 'degree'] == "3차":
        count3.append(data_mer.loc[x,'count'] * 1)
    else: count3.append(data_mer.loc[x,'count'] * 0)
data_mer['count_3'] = list(count3)

In [12]:
count_unknown = []
for x in range(data_mer.shape[0]):
    if data_mer.loc[x, 'degree'] == "확인 중":
        count_unknown.append(data_mer.loc[x,'count'] * 1)
    else: count_unknown.append(data_mer.loc[x,'count'] * 0)
data_mer['count_unknown'] = list(count_unknown)

In [13]:
data_fin = data_mer.groupby('date').apply(max, axis=0)
data_fin = data_fin.drop(['date', 'degree', 'count'], axis=1)
data_fin = data_fin.fillna(0)
print(data_fin)

            count_1  count_2  count_3  count_unknown
date                                                
2020-01-19      1.0      0.0      0.0            0.0
2020-01-20      0.0      0.0      0.0            0.0
2020-01-21      0.0      0.0      0.0            0.0
2020-01-22      1.0      0.0      0.0            0.0
2020-01-23      0.0      0.0      0.0            0.0
2020-01-24      0.0      0.0      0.0            0.0
2020-01-25      1.0      0.0      0.0            0.0
2020-01-26      1.0      0.0      0.0            0.0
2020-01-27      0.0      0.0      0.0            0.0
2020-01-28      0.0      0.0      0.0            0.0
2020-01-29      0.0      0.0      0.0            0.0
2020-01-30      2.0      1.0      0.0            0.0
2020-01-31      1.0      1.0      2.0            0.0
2020-02-01      0.0      1.0      0.0            0.0
2020-02-02      2.0      0.0      1.0            0.0
2020-02-03      1.0      0.0      0.0            0.0
2020-02-04      0.0      0.0      0.0         

In [14]:
# 날짜별 확진자
import pyecharts.options as opts
from pyecharts.charts import Bar, Page
from pyecharts.commons.utils import JsCode
from pyecharts.faker import Collector, Faker

In [17]:
date_patient = (
    Bar()
    .add_xaxis(list(pd.date_range("2020-01-19", "2020-02-17").strftime('%m/%d')))
    .add_yaxis(
        "1차", list(data_fin['count_1']), stack="stack1")
    .add_yaxis(
        "2차", list(data_fin['count_2']), stack="stack1")
    .add_yaxis(
        "3차", list(data_fin['count_3']), stack="stack1")
    .add_yaxis(
        "확인 중", list(data_fin['count_unknown']), stack="stack1")
    .set_colors(["maroon", "firebrick", "orangered", "gray"])
    .set_global_opts(title_opts=opts.TitleOpts(title="날짜별 확진자 수(명)"))               
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))    
)

In [18]:
date_patient.render_notebook()